In [1]:
# pip install pandas

In [2]:
# pip install gensim

In [3]:
# pip install scipy

In [4]:
# pip install pyLDAvis

In [1]:
import pandas as pd
import numpy as np
import glob
from nltk.corpus import stopwords


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
from nltk.corpus import stopwords

# vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
df = pd.read_parquet('umap_df_last_model.parquet')
df

,id,title,abstract,pub_year,authors_list,keyword,relative_date,link,xs,ys
0,thai_antioxidant_rich_food_1,Nutrients value and antioxidant content of ind...,Evidence from epidemiological studies has stro...,2015,"[Rin Charoensiri, Ratchanee Kongkachuichai, Ka...",thai_antioxidant_rich_food,2024-05-17T11:56:28.319188,https://www.researchgate.net/publication/26917...,6.844834,2.516862
1,thai_antioxidant_rich_food_2,Antioxidant and antiaging effect of traditiona...,Objective\nThis study explored the rejuvenatio...,2021,"[Jing Lin, Sutticha Ruangchuay, Qiangqiang Wan...",thai_antioxidant_rich_food,2024-05-17T11:56:56.487936,https://www.researchgate.net/publication/35017...,5.530480,-0.897771
2,thai_antioxidant_rich_food_3,Antioxidant activity of northern and northeast...,Antioxidant capacities were demonstrated betwe...,2006,"[P. Auttaviboonkul, G. Trakoontivakorn, P. Tan...",thai_antioxidant_rich_food,2024-05-17T11:57:35.440265,https://www.researchgate.net/publication/28589...,7.329749,3.032443
3,thai_antioxidant_rich_food_4,Antioxidant and anti-inflammatory activities o...,This study investigated the antioxidant and an...,2012,"[Maitree - Suttajit, Chiang Mai University, Ch...",thai_antioxidant_rich_food,2024-05-17T11:57:53.256914,https://www.researchgate.net/publication/26838...,8.268229,3.846576
4,thai_antioxidant_rich_food_5,Foods from banana inflorescences and their ant...,This study aimed to categorize Thai foods made...,2021,"[Khongsak Srikaeo, Pibulsongkram Rajabhat Univ...",thai_antioxidant_rich_food,2024-05-17T11:58:09.369379,https://www.researchgate.net/publication/35505...,7.096112,3.139917
...,...,...,...,...,...,...,...,...,...,...
1935,HDL_in_foods_230,Theobromine Does Not Affect Fasting and Postpr...,Scope\nChocolate consumption lowers cardiovasc...,2018,"[Lotte Smolders, Ronald P Mensink, Charlotte P...",HDL_in_foods,2024-05-21T17:22:23.580934,https://www.researchgate.net/publication/32535...,3.322277,2.375176
1936,HDL_in_foods_232,Rebuilding the Food Pyramid,Dietary guides introduced in 1992 and 2005 hav...,2003,"[Meir J Stampfer, Walter C Willett]",HDL_in_foods,2024-05-21T17:24:34.862232,https://www.researchgate.net/publication/10970...,3.118114,2.218809
1937,HDL_in_foods_233,Long-term effects of consuming foods containin...,The effects of consuming foods containing 0 (c...,1998,"[J C Kong, L D Dugan, University of Chicago, K...",HDL_in_foods,2024-05-21T17:25:04.547675,https://www.researchgate.net/publication/13734...,3.779132,2.489843
1938,HDL_in_foods_234,Effect of Chromium on Glucose and Lipid Profil...,Purpose: \nChromium (Cr) as an essential trace...,2013,"[Amir Farshchi, Mohammad Abdollahi, Tehran Uni...",HDL_in_foods,2024-05-21T17:25:22.043087,https://www.researchgate.net/publication/23691...,2.908130,2.575327


In [5]:
stopwords = stopwords.words("english")

In [7]:
# print (stopwords)

In [9]:
data = df['title'] + df['abstract']
data[0][0:90]

'Nutrients value and antioxidant content of indigenous vegetables from Southern ThailandEvi'

In [11]:
# Filter for rows with non-null values in both title and abstract
df_filtered = df[~df['title'].isnull() & ~df['abstract'].isnull()]
data = df_filtered['title'] + df_filtered['abstract']

In [13]:
# Load spaCy model (disable unnecessary components for efficiency)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return texts_out

# Lemmatize the combined titles and abstracts
lemmatized_texts = lemmatization(data.tolist())

# Print the first 90 characters of the first lemmatized text
print(lemmatized_texts[0][0:90])

nutrient value antioxidant content indigenous vegetable epidemiological study strongly sug


In [14]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['nutrient', 'value', 'antioxidant', 'content', 'indigenous', 'vegetable', 'epidemiological', 'study', 'strongly', 'suggest', 'diet', 'rich', 'fruit', 'vegetable', 'play', 'vital', 'role', 'disease', 'prevention', 'aim']


In [15]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 2), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 4), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]
acid


In [25]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=38,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [26]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
29     0.293954 -0.408958       1        1  13.837443
5     -0.254406 -0.444468       2        1  13.473031
13    -0.153184 -0.456701       3        1  12.157366
14    -0.260093 -0.327495       4        1   9.674493
3     -0.398919 -0.265127       5        1   7.163334
7     -0.515521 -0.020128       6        1   7.077626
11     0.120137 -0.379506       7        1   5.039863
4     -0.010007 -0.466277       8        1   2.891044
18     0.333837 -0.263066       9        1   2.569712
12     0.167051 -0.485616      10        1   2.352811
30    -0.435657 -0.091005      11        1   2.312653
16    -0.148713  0.519729      12        1   1.990000
9      0.328949  0.350875      13        1   1.948982
25     0.317848 -0.139547      14        1   1.615825
32     0.446945  0.082852      15        1   1.613836
20     0.465746 -0.097794      16        1   1.582770
34    -0.077158 -0.266105      17        1   1.569332
35    -0.377464  0.238378      18        1   1.473214
26     0.120613  0.429276      19        1   1.409113
2     -0.318717  0.084481      20        1   1.268406
27    -0.289992 -0.103814      21        1   1.242582
28     0.341984  0.203804      22        1   1.017776
8     -0.226914  0.364326      23        1   0.852741
33    -0.247480  0.220611      24        1   0.789131
22     0.311499  0.055271      25        1   0.717780
15     0.132509  0.282506      26        1   0.377895
10    -0.083475  0.280328      27        1   0.286103
6      0.105653 -0.076494      28        1   0.281181
23    -0.095456  0.111418      29        1   0.274057
1      0.160187  0.139826      30        1   0.259423
19     0.014017  0.261684      31        1   0.216203
24    -0.042366  0.024189      32        1   0.162919
36     0.108373  0.061614      33        1   0.153972
17     0.025092  0.058215      34        1   0.131855
37     0.007252  0.127289      35        1   0.099960
21     0.069353  0.170775      36        1   0.080628
0      0.035597  0.114024      37        1   0.029018
31     0.028926  0.110627      38        1   0.005921, topic_info=            Term         Freq        Total Category  logprob  loglift
165         food  3359.000000  3359.000000  Default  10.0000  10.0000
439  cholesterol  1488.000000  1488.000000  Default   9.0000   9.0000
0           acid  1381.000000  1381.000000  Default   8.0000   8.0000
707         risk  1274.000000  1274.000000  Default   7.0000   7.0000
19          diet  2730.000000  2730.000000  Default   6.0000   6.0000
..           ...          ...          ...      ...      ...      ...
5          apple     0.001176    74.156026  Topic38  -9.3413  -1.3173
6        benefit     0.001176   509.058002  Topic38  -9.3413  -3.2437
7      bioactive     0.001176   335.340383  Topic38  -9.3413  -2.8263
8       carotene     0.001176    60.388625  Topic38  -9.3413  -1.1120
9         cashew     0.001176     6.417883  Topic38  -9.3413   1.1297

[578 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
73       13  0.986727  accumulation
0         1  0.138982          acid
0         2  0.000724          acid
0         4  0.374963          acid
0         5  0.000724          acid
...     ...       ...           ...
1976     16  0.979808         world
1061      3  0.363757          year
1061     11  0.634391          year
716      14  0.970900        yellow
1408     33  0.808863            µg

[828 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[30, 6, 14, 15, 4, 8, 12, 5, 19, 13, 31, 17, 10, 26, 33, 21, 35, 36, 27, 3, 28, 29, 9, 34, 23, 16, 11, 7, 24, 2, 20, 25, 37, 18, 38, 22, 1, 32])

In [33]:
num_topics = lda_model.num_topics
topics = []

for i in range(num_topics):
    topic_words = lda_model.show_topic(i, topn=10)  # Adjust topn to get more or fewer words
    topics.append([word for word, _ in topic_words])

# Create a DataFrame
df = pd.DataFrame(topics, columns=[f'Word_{i+1}' for i in range(len(topics[0]))], index=[f'Topic_{i+1}' for i in range(num_topics)])

df

,Word_1,Word_2,Word_3,Word_4,Word_5,Word_6,Word_7,Word_8,Word_9,Word_10
Topic_1,nature,ig,rhamnoide,diuretic,flavon,nimbolide,oleanic,tocol,ursolic,biloba
Topic_2,bacteria,bacterial,prebiotic,antibacterial,minimum,pathogenic,coli,bear,derivative,spectrum
Topic_3,plant,specie,family,additional,medicinal,edible,traditional,know,leaf,heavy
Topic_4,health,functional,such,source,benefit,cancer,potential,also,property,include
Topic_5,risk,disease,factor,cardiovascular,associate,consumption,increase,prevention,heart,reduce
Topic_6,group,increase,effect,study,level,control,significantly,decrease,day,weight
Topic_7,fiber,quantitative,proximate,scope,moisture,ash,composition,width,amine,rhamnoide
Topic_8,serum,body,population,effect,role,research,evidence,clinical,review,human
Topic_9,acid,oil,storage,essential,phase,promise,constituent,recovery,wine,analytical
Topic_10,stress,oxidative,resistance,biomarker,disorder,define,model,conventional,molecular,lifespan


In [39]:
num_topics = lda_model.num_topics
words_list = []

# Extract words for each topic
for i in range(num_topics):
    topic_words = lda_model.show_topic(i, topn=10)  # Adjust topn as needed
    for word, prob in topic_words:
        words_list.append({'Topic': f'Topic_{i+1}', 'Word': word})

# Create a DataFrame
df = pd.DataFrame(words_list)

# Save to CSV
df.to_csv('topics_words_foods.csv', index=False)

df

,Topic,Word
0,Topic_1,nature
1,Topic_1,ig
2,Topic_1,rhamnoide
3,Topic_1,diuretic
4,Topic_1,flavon
...,...,...
375,Topic_38,cation
376,Topic_38,reducing
377,Topic_38,thiocyanate
378,Topic_38,shake
